## SITCOM-710: Analysis of TMA velocity, acceleration, and jerk

This notebook is designed to analyze the velocity, acceleration, and jerk of TMA slews. 

The requirements for SITCOM-710 are listed in LTS-103: 2.2.2 Slewing Rates. The values found there are used in the `el_limit_dict` and `az_limit_dict` parameters. 

The bulk of the analysis is done with the file `sitcom710.py`, which contains the class `SlewData` and methods for plotting (`plot_max_hist` and `slew_profile_plot`). 

`SlewData` queries the EFD for `actualPosition`, `actualVelocity`, and `timestamp` for both azimuth and elevation for individual slews, given a range of dayObs. The data from each slew is fit using a spline or savgol model. After the fits are calculated, the absolute maximums of the velocity, acceleration, and jerk for the fits of each slew are identified and compiled together into their own dataframe. The data can be accessed through a `SlewData` object.

**Note**: This notebook currently requires the `sitcom-performance-analysis` branch of the `summit_utils` repo (at least until `relatesTo()` is in the develop branch)

Current plots:
- The position, velocity, acceleration, and jerk over the time period of a single slew for both azimuth and elevation
- Histogram of maximums (velocity, acceleration, and jerk) of each of the slews within a choosen range of dayObs


### Running this notebook

These are the parameters that can be changed as needed within the notebook.

For data:
- `begin_day_obs`: The first dayObs of your query.
- `end_day_obs`: The last dayObs of your query.
- `client = EfdClient("idf_efd")`: The parameter controls which copy of the EFD you are querying. Some examples include "usdf_efd", "summit_efd", and "idf_efd".
- `spline_filter`: Two options can be entered, either "spline" or "savgol". The "spline" option uses Univariate splines to calculate the fits, while "savgol" uses a scipy savgol_filter.
- `padding`: The time in seconds that is added/removed from the start and end of a slew. For instance, if you set the padding to 5, this will add 5 seconds of data before and after the duration of the slew. A negative value will remove data.
- `smoothing`: The smoothing parameter used in the spline fit. 
- `kernel_size`: The kernel size used in the spline fit. 
- `block_id`: If you want slews related to a specific block, set this to the block ID number

For plotting:
- `sitcom710.plot_max_hist(...)`
    - `max_frame`: the maximums of each slew generated by `SlewData`
    - `limitsBool`: when set to true will add the max and design spec limits
    - `logBool`: controls whether the y-axis is log scale. 
    - `fit`: describes what fit is used on the data in the title
    - `padding`: describes how much padding is used on the data in the title
- `sitcom710.slew_profile_plot(...)`
    - `actual_az_frame`: measured azimuth data from the EFD query
    - `actual_el_frame`: measured elevation data from the EFD query
    - `spline_frame`: the fit data generated by `SlewData`
    - `dayObs`: the dayObs of the slew you are interested in plotting
    - `slew_index`: the index of the slew you are interested in plotting
    - `limitsBool`: when set to true will add the max and design spec limits

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker

from lsst.sitcom import vandv
import lsst.sitcom.vandv.tma.sitcom710 as sitcom710

import logging

summit_utils_logger = logging.getLogger("lsst.summit.utils")
summit_utils_logger.setLevel(logging.ERROR)

%matplotlib inline
%load_ext lab_black
%load_ext autoreload
%autoreload 2

## Defining parameters

In [ ]:
# included for quick reference
# define limits from science requirements document (LTS-103 2.2.2) for plotting
# units in deg/s - deg/s^2 - deg/s^3
el_limit_dict = {
    "max_velocity": 5.25,
    "max_acceleration": 5.25,
    "max_jerk": 21,
    "design_velocity": 3.5,
    "design_acceleration": 3.5,
    "design_jerk": 14,
}
az_limit_dict = {
    "max_velocity": 10.5,
    "max_acceleration": 10.5,
    "max_jerk": 42,
    "design_velocity": 7,
    "design_acceleration": 7,
    "design_jerk": 28,
}

Below are the parameters that can be changed in the SlewData query. For the purposes of this notebook, some optional parameters will be left at their default value and commented out, but can be implemented for experimenting with the notebook.

In [ ]:
begin_day_obs = 20231116
end_day_obs = 20231116
spline_filter = "spline"
padding = 2
# smoothing = 0.0
# kernel_size = 100
# block_id = -1 # when set to -1, does not query for block IDs

# Query data and run analysis

Store all queried data in a single variable from the days defined above. The following data frames can be called:
- `data.real_az_data`: returns the measured data in azimuth
- `data.real_el_data`: returns the measured data in elevation
- `data.all_data`: returns all of the fitted spline data (both azimuth and elevation)
- `data.max_data`: returns maximum velocity, acceleration, and jerk of each slew in both azimuth and elevation

In [ ]:
client = EfdClient("usdf_efd")
event_maker = TMAEventMaker(client)
data = sitcom710.SlewData(
    begin_day_obs,
    end_day_obs,
    event_maker,
    spline_filter,
    padding=padding,
)
print("Final number of slews: " + str(data.max_data.shape[0]))

## Plots

Identify the maximums of the velocity, acceleration, and jerk of each slew and plot statistics

In [ ]:
sitcom710.plot_max_hist(
    max_frame=data.max_data,
    limitsBool=False,
    logBool=True,
    fit=spline_filter,
    padding=padding,
)

Pick a slew to show a single slew motion analysis plot, first without the spec limits

In [ ]:
sitcom710.slew_profile_plot(
    actual_az_frame=data.real_az_data,
    actual_el_frame=data.real_el_data,
    spline_frame=data.all_data,
    dayObs=20231116,
    slew_index=19,
    limitsBool=False,
)

Let's try the same plot, now including the spec limits

In [ ]:
sitcom710.slew_profile_plot(
    actual_az_frame=data.real_az_data,
    actual_el_frame=data.real_el_data,
    spline_frame=data.all_data,
    dayObs=20231116,
    slew_index=19,
    limitsBool=True,
)